Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd
rat = pd.read_csv('C:\\Users\\Мария\\Desktop\\ml-latest-small\\ratings.csv' ,  encoding='utf-8')
rat.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
rat.groupby('userId').count()[['movieId', 'rating', 'timestamp']].head()


,movieId,rating,timestamp
userId,,,
1,20,20,20
2,76,76,76
3,51,51,51
4,204,204,204
5,100,100,100


In [5]:
ratings_gr = rat.groupby('userId').agg({'timestamp': ['min', 'max'], 'rating': 'count'})
ratings_gr.head()

timestamp             rating
               min         max  count
userId                               
1       1260759108  1260759205     20
2        835355395   835356246     76
3       1298861589  1298932787     51
4        949778714   949982274    204
5       1163373044  1163375145    100

In [6]:
ratings_gr['life_time'] = ratings_gr['timestamp']['max'] - ratings_gr['timestamp']['min']
ratings_gr.head()

timestamp             rating life_time
               min         max  count          
userId                                         
1       1260759108  1260759205     20        97
2        835355395   835356246     76       851
3       1298861589  1298932787     51     71198
4        949778714   949982274    204    203560
5       1163373044  1163375145    100      2101

In [9]:
ratings_gr = ratings_gr[ratings_gr.rating.values >= 100]
ratings_gr.head()

timestamp             rating  life_time
               min         max  count           
userId                                          
4        949778714   949982274    204     203560
8       1154389340  1154474527    116      85187
15       997937239  1469330735   1700  471393496
17      1127468587  1127476640    363       8053
19       855190091   855195373    423       5282

Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [20]:
import pandas as pd
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [21]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [22]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [13]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [24]:
revenue_by_client = rzd.merge(auto, how='outer').merge(air, how='outer')
revenue_by_client #таблица с тремя типами выручки для каждого client_id



,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [42]:
revenue_by_address = client_base.merge(rzd, how='outer').merge(auto, how='outer').merge(air, how='outer')
revenue_by_address.loc[:, 'address':'air_revenue'] # таблица по типам выручки с указанием адреса клиента


,address,rzd_revenue,auto_revenue,air_revenue
0,Комсомольская 4,1093.0,NaN,NaN
1,Энтузиастов 8а,2810.0,NaN,NaN
2,Левобережная 1а,10283.0,57483.0,NaN
3,Мира 14,5774.0,83.0,NaN
4,ЗЖБИиДК 1,981.0,912.0,81.0
5,Строителей 18,NaN,4834.0,4.0
6,Панфиловская 33,NaN,98.0,13.0
7,Мастеркова 4,NaN,NaN,173.0


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? 
Для составления ответа можно использовать вопросы:
1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [57]:
date = { 'user_id' : [1, 1, 1, 2, 2, 3, 4] , 
       'coordinate' : ['554521/373704','554521/373704','554521/373704' ,'533521/373704',' 552321/373434','512521/343704','554671/373674' ], 
       'date' : ['01.01.2020', '02.01.2020', '03.01.2020' , '02.01.2020','07.01.2020', '01.03.2020', '01.04.2020']}
base = pd.DataFrame.from_dict(date)
base.index.name = 'visit_id'
base


,user_id,coordinate,date
visit_id,,,
0,1,554521/373704,01.01.2020
1,1,554521/373704,02.01.2020
2,1,554521/373704,03.01.2020
3,2,533521/373704,02.01.2020
4,2,552321/373434,07.01.2020
5,3,512521/343704,01.03.2020
6,4,554671/373674,01.04.2020


In [ ]:
Я не очень понял суть задания, в качестве работы попытался создать подобный датафрейм. Но как на него наложить ваши вопросы,
я совершенно не понимаю. Прошу вас дать обратную связь, желательно в Slack т.к. будут встречные вопросы. Спасибо!